In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
import string

# Read recipe inputs
ACCOUNTS_WITH_BUNDLER_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_BUNDLER_AND_DUNS")
ACCOUNTS_WITH_BUNDLER_AND_DUNS_df = ACCOUNTS_WITH_BUNDLER_AND_DUNS.get_dataframe()

ACCOUNTS_WITH_EBX_PARTY = dataiku.Dataset("ACCOUNTS_WITH_EBX_PARTY")
ACCOUNTS_WITH_EBX_PARTY_df = ACCOUNTS_WITH_EBX_PARTY.get_dataframe()

MDM_FINAL = dataiku.Dataset("mdm_final")
MDM_FINAL_df = MDM_FINAL.get_dataframe()

MATCHES_VERIFIED = dataiku.Dataset("VERIFIED_MATCHES")
MATCHES_VERIFIED_df = MATCHES_VERIFIED.get_dataframe()

RDW_CONVERSIONS = dataiku.Dataset("NAFCUSTOMER_RDW_CONVERSIONS")
RDW_CONVERSIONS_df = RDW_CONVERSIONS.get_dataframe()

ACCOUNT_NEW_SALES = dataiku.Dataset("ACCOUNT_NEW_SALES")
ACCOUNT_NEW_SALES_df = ACCOUNT_NEW_SALES.get_dataframe()

ACCOUNT_NEW_SALES_FULL = dataiku.Dataset("ACCOUNT_NEW_SALES_FULL")
ACCOUNT_NEW_SALES_FULL_df = ACCOUNT_NEW_SALES_FULL.get_dataframe()

/data/dataiku/dss_data/code-envs/python/env_clc/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/data/dataiku/dss_data/code-envs/python/env_clc/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (1,20,27) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/data/dataiku/dss_data/code-envs/python/env_clc/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [4]:
# load matches calculated by matching process, then verified with secondary matching step

#df_matches_verified = MATCHES_VERIFIED_df
#df_matches_verified["CUSTOMER"] = df_matches_verified['CUSTOMER'].str.translate(str.maketrans('', '', string.punctuation))
#df_matches_verified["CUSTOMER_CLC"] = df_matches_verified['CUSTOMER_CLC'].str.translate(str.maketrans('', '', string.punctuation))
#df_matches_verified.head()

In [2]:
print(len(RDW_CONVERSIONS_df))
RDW_CONVERSIONS_df.head()

2151579


,FLEET_ID,CLASSIC_ACCOUNT_NUMBER,STATUS_INDICATOR,FLEET_NAME,CRDT_LMT_AMT,REP,GROUP_CD,STATUS_DATE,SETUP_DATE,DAYS_SINCE_SETUP,ACCOUNT_AGE
0,9100004780710,NaN,Terminated,S R W Cooling & Heating,1200,10900755,NaN,2018-07-20 00:00:00.000,2018-03-05 00:00:00.000,1718,56
1,0406008243024,NaN,Terminated,GOING IN STYLE LIMOUSINE SERVI,18000,WU1,NaN,2020-06-22 00:00:00.000,2010-05-11 00:00:00.000,4573,150
2,9100000783477,NaN,Terminated,Special Needs Program (Nid.),0,NaN,NaN,2016-12-20 00:00:00.000,2009-11-13 00:00:00.000,4752,156
3,9100003622680,NaN,Terminated,National Refurbishing Center Inc,3000,NaN,NaN,2016-06-22 00:00:00.000,2015-12-04 00:00:00.000,2540,83
4,9100008835498,NaN,Terminated,Grace Z Investments LLC,500,NaN,NaN,2021-10-28 00:00:00.000,2021-02-21 00:00:00.000,634,21


In [3]:
# load MDM matches, note we are using an older MDM Final extract from March
# TODO: replace MDM final with snowflake source

# prepare mdm final output:
# retain key columns
# cast ids to large ints
# remove punctuation cast names upper case
# filter out invalid grouping entries

df_mdm = MDM_FINAL_df[['accountnumber','wex_id','name','global_customer_id','global_customer_name']].copy()
df_mdm.columns = ['CUSTOMER_ACCOUNT_ID','MDM_WEX_ID','MDM_WEX_NAME','MDM_PARTY_ID','MDM_PARTY_NAME']
df_mdm.dropna(subset=['CUSTOMER_ACCOUNT_ID'], inplace=True)
df_mdm['CUSTOMER_ACCOUNT_ID'] = df_mdm['CUSTOMER_ACCOUNT_ID'].astype('Int64', errors='ignore')
df_mdm['MDM_WEX_ID'] = df_mdm['MDM_WEX_ID'].astype('Int64', errors='ignore')
df_mdm['MDM_PARTY_ID'] = df_mdm['MDM_PARTY_ID'].astype('Int64', errors='ignore')
print(len(df_mdm))

df_mdm['MDM_WEX_NAME'] = df_mdm['MDM_WEX_NAME'].str.upper()
df_mdm["MDM_WEX_NAME"] = df_mdm['MDM_WEX_NAME'].str.translate(str.maketrans('', '', string.punctuation))

print(len(df_mdm))
df_mdm = df_mdm[df_mdm.MDM_WEX_NAME!='CARD TYPE 7 PRIMARY']
df_mdm = df_mdm[df_mdm.MDM_WEX_NAME!='ELEMENT 1']
df_mdm = df_mdm[df_mdm.MDM_WEX_NAME!='ELEMENT 2']
print(len(df_mdm))

df_mdm['MDM_PARTY_NAME'] = df_mdm['MDM_PARTY_NAME'].str.upper()
df_mdm["MDM_PART_NAME"] = df_mdm['MDM_PARTY_NAME'].str.translate(str.maketrans('', '', string.punctuation))
df_mdm.head()

1915322
1915322
1912902


,CUSTOMER_ACCOUNT_ID,MDM_WEX_ID,MDM_WEX_NAME,MDM_PARTY_ID,MDM_PARTY_NAME,MDM_PART_NAME
0,1073,11916901,WELDWOOD OF CANADA LTD,11916901,WELDWOOD OF CANADA LTD,WELDWOOD OF CANADA LTD
1,1177,9502492,PROVOST CARTAGE INC,9502492,PROVOST CARTAGE INC,PROVOST CARTAGE INC
2,1826,9502609,TOM MCCALL PONTIAC BUICK LTD,9502609,TOM MCCALL PONTIAC BUICK LTD,TOM MCCALL PONTIAC BUICK LTD
3,2157,9502685,DANIELS AND MANNARD LIMITED,9502685,DANIELS AND MANNARD LIMITED,DANIELS AND MANNARD LIMITED
4,2505,9502757,TERRATECH EQUIPMENT INC,9502757,TERRATECH EQUIPMENT INC,TERRATECH EQUIPMENT INC


In [4]:
# create structure at account level with MDM WEX ID and MDM WEX NAME
# that can be joined to source accounts

df_wex_id = df_mdm.groupby(['MDM_WEX_ID', 'MDM_WEX_NAME'])[['CUSTOMER_ACCOUNT_ID']].count().reset_index()
df_wex_id.columns = ['MDM_WEX_ID','MDM_WEX_NAME','COUNT_OF_ACCOUNT']
df_wex_id = df_wex_id[df_wex_id.COUNT_OF_ACCOUNT>1]
df_wex_id['HAS_WEX_ID'] = True
df_wex_id.sort_values(by=['COUNT_OF_ACCOUNT'],ascending=False, inplace=True)

df_wex_id.tail()
print(len(df_wex_id))
df_wex_id.head()

df_account_with_wex_id = df_mdm[['CUSTOMER_ACCOUNT_ID','MDM_WEX_ID']].copy()
df_account_with_wex_id.dropna(subset=['MDM_WEX_ID'], inplace=True)
print(len(df_account_with_wex_id))
df_account_with_wex_id = pd.merge(df_account_with_wex_id, df_wex_id, on='MDM_WEX_ID', how='left')
print(len(df_account_with_wex_id))
df_account_with_wex_id.head()
df_account_with_wex_id.dropna(subset=['HAS_WEX_ID'], inplace=True)
df_account_with_wex_id.drop_duplicates(subset=['CUSTOMER_ACCOUNT_ID'], inplace=True)
df_account_with_wex_id = df_account_with_wex_id[['CUSTOMER_ACCOUNT_ID','MDM_WEX_ID','MDM_WEX_NAME']]

print(len(df_account_with_wex_id))
df_account_with_wex_id.head()

269078
1906558
2872944
802585


,CUSTOMER_ACCOUNT_ID,MDM_WEX_ID,MDM_WEX_NAME
0,1073,11916901,WELDWOOD OF CANADA LTD
4,2505,9502757,TERRATECH EQUIPMENT INC
5,2547,11916960,TRA FOODS LTD
6,2645,9502792,THE WAWANESA MUTUAL INSURANCE COMPAN
7,2650,9502797,THE WAWANESA MUTUAL INSURANCE COMPAN


In [5]:
# Join accounts with MDM grouping to accounts
# ensure that counts before and after match, there should be no row duplication
df = ACCOUNTS_WITH_BUNDLER_AND_DUNS_df.copy()
print(len(df))

df_account_with_wex_id['CUSTOMER_ACCOUNT_ID'] = df_account_with_wex_id['CUSTOMER_ACCOUNT_ID'].astype('Int64', errors='ignore')
df = pd.merge(df,df_account_with_wex_id, on='CUSTOMER_ACCOUNT_ID', how='left')
print(len(df))
df.head()

882017
882017


,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,DUNS,IS_BUNDLER,DNB_GLOBAL_ULT_NUMBER,DNB_GLOBAL_ULT_NAME,DNB_DOMESTIC_ULT_NUMBER,DNB_DOMESTIC_ULTIMATE_NAME,DNB_HQ_NUMBER,DNB_HQ_NAME,DNB_DUNS_NUMBER,DNB_BUSINESS_NAME,MDM_WEX_ID,MDM_WEX_NAME
0,6133004391652,UNITED RANGER INC 61,UNITED RANGER INC 61,48436943.0,False,NaN,NaN,NaN,NaN,NaN,NaN,48436943.0,"United Ranger, Inc.",<NA>,NaN
1,9100005747877,UNITED ELECTRIC,NaN,48436943.0,True,NaN,NaN,NaN,NaN,NaN,NaN,48436943.0,"United Ranger, Inc.",<NA>,NaN
2,434006814024,SUPERIOR MOVING SYSTEM (27703),SUPERIOR MOVING SYSTEM (27703),961258225.0,False,NaN,NaN,NaN,NaN,NaN,NaN,961258225.0,"Superior Moving Systems, Inc.",11942872,SUPERIOR MOVING SYSTEM 27703
3,9100009665847,SUPERIOR MOVING SYSTEM 27703,NaN,961258225.0,True,NaN,NaN,NaN,NaN,NaN,NaN,961258225.0,"Superior Moving Systems, Inc.",11942872,SUPERIOR MOVING SYSTEM 27703
4,9100007495371,NORTH AMHERST MOTORS INC,NaN,66974098.0,True,NaN,NaN,NaN,NaN,NaN,NaN,66974098.0,"North Amherst Motors, Inc.",11797339,NORTH AMHERST MOTORS INC


In [0]:
import warnings
warnings.filterwarnings(action='once')

df['DUNS'] = df['DUNS'].astype('Int64', errors='ignore')
df['DNB_DUNS_NUMBER'] = df['DNB_DUNS_NUMBER'].astype('Int64', errors='ignore')
df['DNB_BUSINESS_NAME'] = df['DNB_BUSINESS_NAME'].str.upper()
df["DNB_BUSINESS_NAME"] = df['DNB_BUSINESS_NAME'].str.translate(str.maketrans('', '', string.punctuation))

df['DNB_GLOBAL_ULT_NUMBER'] = df['DNB_GLOBAL_ULT_NUMBER'].astype('Int64', errors='ignore')
df['DNB_GLOBAL_ULT_NAME'] = df['DNB_GLOBAL_ULT_NAME'].str.upper()
df["DNB_GLOBAL_ULT_NAME"] = df['DNB_GLOBAL_ULT_NAME'].str.translate(str.maketrans('', '', string.punctuation))

df['DNB_DOMESTIC_ULT_NUMBER'] = df['DNB_DOMESTIC_ULT_NUMBER'].astype('Int64', errors='ignore')
df['DNB_DOMESTIC_ULTIMATE_NAME'] = df['DNB_DOMESTIC_ULTIMATE_NAME'].str.upper()
df["DNB_DOMESTIC_ULTIMATE_NAME"] = df['DNB_DOMESTIC_ULTIMATE_NAME'].str.translate(str.maketrans('', '', string.punctuation))

df['DNB_HQ_NUMBER'] = df['DNB_HQ_NUMBER'].astype('Int64', errors='ignore')
df['DNB_HQ_NAME'] = df['DNB_HQ_NAME'].str.upper()
df["DNB_HQ_NAME"] = df['DNB_HQ_NAME'].str.translate(str.maketrans('', '', string.punctuation))

df['DNB_LEVEL'] = 'None'
df['DNB_CUSTOMER_NAME'] = np.nan

df.loc[~df["DNB_GLOBAL_ULT_NAME"].isnull(),'DNB_LEVEL'] = "DUNS Global"
df.loc[~df["DNB_GLOBAL_ULT_NAME"].isnull(),'DNB_CUSTOMER_NAME'] = df.DNB_GLOBAL_ULT_NAME

df.loc[(df["DNB_GLOBAL_ULT_NAME"].isnull())&(~df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull()),'DNB_LEVEL'] = "DUNS Domestic"
df.loc[(df["DNB_GLOBAL_ULT_NAME"].isnull())&(~df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_DOMESTIC_ULTIMATE_NAME

df.loc[(df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull())&(~df["DNB_HQ_NAME"].isnull()),'DNB_LEVEL'] = "DUNS HQ"
df.loc[(df["DNB_DOMESTIC_ULTIMATE_NAME"].isnull())&(~df["DNB_HQ_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_HQ_NAME

df.loc[(df["DNB_HQ_NAME"].isnull())&(~df["DNB_BUSINESS_NAME"].isnull()),'DNB_LEVEL'] = "DUNS"
df.loc[(df["DNB_HQ_NAME"].isnull())&(~df["DNB_BUSINESS_NAME"].isnull()),'DNB_CUSTOMER_NAME'] = df.DNB_BUSINESS_NAME

df['EDW_STATE'] = 'Unknown'
df.loc[df["EDW_CUSTOMER_NAME"].isnull(),'EDW_STATE'] = "None"
df.loc[~df["EDW_CUSTOMER_NAME"].isnull(),'EDW_STATE'] = "Set"

df['EDW_CUSTOMER_NAME_ORIGINAL'] = df['EDW_CUSTOMER_NAME']
df['CUSTOMER_ACCOUNT_NAME_ORIGINAL'] = df['CUSTOMER_ACCOUNT_NAME']

df['EDW_CUSTOMER_NAME'].str.strip()
ending_tokens = [' 2', ' 3', ' 4', ' 04', ' 5', ' 6', ' 7', ' 8', ' 9',' (2)',
                 ' (3)',' (04)',' (4)', ' (5)', ' (6)', ' (7)', ' (8)',
                 ' (9)',' (25)','  (32)', ' AD', ' LD']

for s in ending_tokens:
    index_offset = -1*(len(s))
    df.loc[df['EDW_CUSTOMER_NAME'].str.endswith(s, na=False),"EDW_CUSTOMER_NAME"] = df['EDW_CUSTOMER_NAME'].str[:index_offset]

df['CUSTOMER_ACCOUNT_NAME'].str.strip()
for s in ending_tokens:
    index_offset = -1*(len(s))
    df.loc[df['CUSTOMER_ACCOUNT_NAME'].str.endswith(s, na=False),"CUSTOMER_ACCOUNT_NAME"] = df['CUSTOMER_ACCOUNT_NAME'].str[:index_offset]

df['CUSTOMER'] = np.nan
df['CUST_CALC_SOURCE'] = 'Unknown'
df['CUST_CALC_RULE'] = 'None'
df.loc[~df["EDW_CUSTOMER_NAME"].isnull(),'CUSTOMER'] = df["EDW_CUSTOMER_NAME"]
df.loc[~df["EDW_CUSTOMER_NAME"].isnull(),'CUST_CALC_SOURCE'] = "EDW"

df.loc[(df["CUSTOMER"].isnull())&(~df["DNB_CUSTOMER_NAME"].isnull()),'CUST_CALC_SOURCE'] = "DNB"
df.loc[df["CUSTOMER"].isnull(),'CUSTOMER'] = df["DNB_CUSTOMER_NAME"]

df.loc[df['CUSTOMER'].isnull(), 'CUST_CALC_SOURCE'] = 'MDM'
df.loc[df['CUSTOMER'].isnull(), 'CUSTOMER'] = df['MDM_WEX_NAME']

df.loc[df["CUSTOMER"].isnull(),'CUST_CALC_SOURCE'] = 'ACCOUNT'
df.loc[df["CUSTOMER"].isnull(),'CUSTOMER'] = df["CUSTOMER_ACCOUNT_NAME"]

# RULE SETs
def apply_rule(df, rule_name,filter_name_list,final_name):

    df.loc[df['CUSTOMER'].isin(filter_name_list),"CUST_CALC_SOURCE"] = "CUSTOM RULE"
    df.loc[df['CUSTOMER'].isin(filter_name_list),"CUST_CALC_RULE"] = rule_name
    df.loc[df['CUSTOMER'].isin(filter_name_list),"CUSTOMER"] = final_name

    return(df)

def apply_rule_starts_with(df, rule_name, compares_to, starts_with_string,final_name):

    df.loc[df[compares_to].str.startswith(starts_with_string, na=False),"CUST_CALC_SOURCE"] = "CUSTOM RULE"
    df.loc[df[compares_to].str.startswith(starts_with_string, na=False),"CUST_CALC_RULE"] = rule_name
    df.loc[df[compares_to].str.startswith(starts_with_string, na=False),"CUSTOMER"] = final_name

    return(df)

def apply_rule_contains(df, rule_name, compares_to, contains_string,final_name):

    df.loc[df[compares_to].str.contains(contains_string, na=False),"CUST_CALC_SOURCE"] = "CUSTOM RULE"
    df.loc[df[compares_to].str.contains(contains_string, na=False),"CUST_CALC_RULE"] = rule_name
    df.loc[df[compares_to].str.contains(contains_string, na=False),"CUSTOMER"] = final_name

    return(df)

# The following set of ~100 rules represents high card count mappings
# these rules have been manually verified
# as they impact large numbers of cards (and in turn gallons, spend and revenue)

df = apply_rule(df, "RULE 001", ['QUANTA SERVICES INC','QUANTA SERVICES'], 'QUANTA SERVICES INC')
df = apply_rule(df, "RULE 002", ['7325 ADVANCE STORES COMPANY 4','7325 ADVANCE AUTO','7325 ADVANCE STORES COMP','7325 ADVANCE STORES COMP 2'], 'ADVANCE AUTO')
df = apply_rule(df, "RULE 003", ['17435-GE HEALTHCARE (3)','17435 GE HEALTHCARE','17435-GE HEALTHCARE','17435-GE HEALTHCARE (2)'], '17435-GE HEALTHCARE')
df = apply_rule(df, "RULE 004", ['ADAPTHEALTH CORP','ADAPTHEALTH LLC'], 'ADAPTHEALTH')
df = apply_rule(df, "RULE 005", ['AGL RESOURCES 5CU6','AGL RESOURCES ARI'], 'AGL RESOURCES')
df = apply_rule(df, "RULE 006", ['APTIVE ENVIRONMENTAL LLC','ADAPTIVE ENVIRONMENTAL CONSULTING INC','APTIVE ENVIRONMENTAL LLC','APTIVE ENVIRONMENTAL 0CV7'], 'APTIVE ENVIRONMENTAL')
df = apply_rule(df, "RULE 007", [' MMCCARTHY TIRE','MCCARTHY TIRE'], 'MCCARTHY TIRE')
df = apply_rule(df, "RULE 008", ['FOSS NATIONAL/CORP-RATEFOSS NATIONAL LEASING (2)','FOSS NATIONAL LEASING (2)'], 'FOSS NATIONAL LEASING')
df = apply_rule(df, "RULE 009", ['JOHNSONJOHNSON','JOHNSON JOHNSON CITRUS'], 'JOHNSON JOHNSON')
df = apply_rule(df, "RULE 010", ['HELMERICH AND PAYNE INC PARENT','HELMERICH  PAYNE ID 5FM8'], 'HELMERICH AND PAYNE INC')
df = apply_rule(df, "RULE 011", ['LABCORP','LABCORP (3LAB)','LABCORP (3LAB)(2)','LABORATORY CORPORATION OF AMERICA'], 'LABORATORY CORPORATION OF AMERICA')
df = apply_rule(df, "RULE 012", ['LEHIGH HANSON (3LHN)','LEHIGH HANSON (3LHN)(2)','3995 LEHIGH HANSON INC','116710-LEHIGH HANSON, INC (2)',
                                 '116710-LEHIGH HANSON, INC (3)','116710-LEHIGH HANSON, INC (4)','116710-LEHIGH HANSON, INC (6)',
                                 "E450 LEHIGH HANSON MATERIALS L",'LEHIGH HANSON INC3LHC'], 'LEHIGH HANSON INC')
df = apply_rule(df, "RULE 013", ['NOVONORDISK','NOVO NORDISK','NOVO NORDISK INC','NOVO NORDISK FONDEN'], 'NOVO NORDISK INC')
df = apply_rule(df, "RULE 014", ['US LBM (5EE8)','US LBM (5EE8)(2)','US LBM HOLDINGS, LLC (5EE8)','LAMPERT YARDS US LBM LLC','US LBM HOLDINGS LLC 5EE8'], 'US LBM HOLDINGS LLC')
df = apply_rule(df, "RULE 015", ['VEOLIA WATER LOGISTICS (2R63)','VEOLIA LOGISTICS 2R63'], 'VEOLIA LOGISTICS')

df = apply_rule_starts_with(df, "RULE 016",'CUSTOMER',"AT  T" , "AT&T")
df = apply_rule_starts_with(df, "RULE 017",'CUSTOMER',"FEDEX" , "FEDEX")
df = apply_rule_starts_with(df, "RULE 018",'CUSTOMER','ARAMARK', 'ARAMARK')
df = apply_rule_starts_with(df, "RULE 019",'CUSTOMER','CABLE ONE', 'CABLE ONE INC')
df = apply_rule_starts_with(df, "RULE 020",'CUSTOMER','CBRE', 'CBRE GROUP INC')
df = apply_rule_starts_with(df, "RULE 021",'CUSTOMER','COMCAST CABLE', 'COMCAST CABLE')
df = apply_rule_starts_with(df, "RULE 022",'CUSTOMER','COMPASS GROUP', 'COMPASS GROUP')
df = apply_rule_starts_with(df, "RULE 023",'CUSTOMER','CONOCOPHILLIPS', 'CONOCOPHILLIPS')
df = apply_rule_starts_with(df, "RULE 024",'CUSTOMER','CSC SERVICEWORKS', 'CSC SERVICEWORKS')
df = apply_rule_starts_with(df, "RULE 025",'CUSTOMER','CROWN CASTLE USA', 'CROWN CASTLE USA')
df = apply_rule_starts_with(df, "RULE 026",'CUSTOMER','E JOHNSON CONTROLS', 'E JOHNSON CONTROLS FIRE & SEC')
df = apply_rule_starts_with(df, "RULE 027",'CUSTOMER','FASTENAL', 'FASTENAL COMPANY')
df = apply_rule_starts_with(df, "RULE 028",'CUSTOMER','GENERAL MILLS', 'GENERAL MILLS')
df = apply_rule_starts_with(df, "RULE 029",'CUSTOMER','J R SIMPLOT', 'J R SIMPLOT')
df = apply_rule_starts_with(df, "RULE 030",'CUSTOMER','JC EHRLICH', 'JC EHRLICH')
df = apply_rule_starts_with(df, "RULE 031",'CUSTOMER','KINDER MORGAN', 'KINDER MORGAN')
df = apply_rule_starts_with(df, "RULE 032",'CUSTOMER','LIBERTY MUTUAL', 'LIBERTY MUTUAL')
df = apply_rule_starts_with(df, "RULE 033",'CUSTOMER','IGT GLOBAL', 'IGT GLOBAL')
df = apply_rule_starts_with(df, "RULE 034",'CUSTOMER','MARATHON PETROLEUM', 'MARATHON PETROLEUM')
df = apply_rule_starts_with(df, "RULE 035",'CUSTOMER','MONDELEZ GLOBAL', 'MONDELEZ GLOBAL')
df = apply_rule_starts_with(df, "RULE 036",'CUSTOMER','NATIONAL FUEL', 'NATIONAL FUEL')
df = apply_rule_starts_with(df, "RULE 037",'CUSTOMER','NEXSTAR BROADCASTING', 'NEXSTAR BROADCASTING')
df = apply_rule_starts_with(df, "RULE 038",'CUSTOMER','NORFOLK SOUTHERN', 'NORFOLK SOUTHERN')
df = apply_rule_starts_with(df, "RULE 039",'CUSTOMER','NORTHERN CLEARING', 'NORTHERN CLEARING')
df = apply_rule_starts_with(df, "RULE 040",'CUSTOMER','PHILLIPS 66 COMPANY', 'PHILLIPS 66 COMPANY')
df = apply_rule_starts_with(df, "RULE 041",'CUSTOMER','SCHINDLER ELEVATOR', 'SCHINDLER ELEVATOR')
df = apply_rule_starts_with(df, "RULE 042",'CUSTOMER','STONEMOR', 'STONEMOR')
df = apply_rule_starts_with(df, "RULE 043",'CUSTOMER','SYNGENTA', 'SYNGENTA')
df = apply_rule_starts_with(df, "RULE 044",'CUSTOMER','TRANSDEV', 'TRANSDEV')
df = apply_rule_starts_with(df, "RULE 045",'CUSTOMER','UNITED RENTALS', 'UNITED RENTALS INC')
df = apply_rule_starts_with(df, "RULE 046",'CUSTOMER','VAN POOL TRANSPORTATION', 'VAN POOL TRANSPORTATION')
df = apply_rule_starts_with(df, "RULE 047",'CUSTOMER','WILLIAMS STRATEGIC', 'WILLIAMS STRATEGIC')
df = apply_rule_starts_with(df, "RULE 048",'CUSTOMER','XTO ENERGY', 'XTO ENERGY')
df = apply_rule_starts_with(df, "RULE 049",'CUSTOMER',"BIMBO" , "BIMBO BAKERIES USA INC")

df = apply_rule_starts_with(df, "RULE 050",'CUSTOMER',"MANSFIELD OIL" , "MANSFIELD OIL")
df = apply_rule_starts_with(df, "RULE 051",'CUSTOMER',"ASPLUNDH" , "ASPLUNDH")
df = apply_rule_starts_with(df, "RULE 052",'CUSTOMER',"CINTAS CORPORATION" , "CINTAS CORPORATION")
df = apply_rule_starts_with(df, "RULE 053",'CUSTOMER',"ENTERPRISE RAC" , "ENTERPRISE RAC")
df = apply_rule_starts_with(df, "RULE 054",'CUSTOMER',"THE CRAWFORD GROUP INC" , "ENTERPRISE RAC")
df = apply_rule_starts_with(df, "RULE 055",'DNB_CUSTOMER_NAME',"STATE OF NEW YORK" , "STATE OF NEW YORK")
df = apply_rule_starts_with(df, "RULE 056",'DNB_CUSTOMER_NAME',"STATE OF GEORGIA" , "STATE OF GEORGIA")
df = apply_rule_starts_with(df, "RULE 057",'DNB_CUSTOMER_NAME',"VERIZON COMMUNICATIONS INC" , "VERIZON SOURCING LLC")
df = apply_rule_starts_with(df, "RULE 058",'DNB_CUSTOMER_NAME',"STATE OF NORTH CAROLINA" , "STATE OF NORTH CAROLINA")
df = apply_rule_starts_with(df, "RULE 059",'CUSTOMER',"DYCOM INDUSTRIES" , "DYCOM INDUSTRIES")
df = apply_rule_starts_with(df, "RULE 060",'CUSTOMER',"TERMINIX" , "TERMINIX CONSUMER SERVICES LLC")
df = apply_rule_starts_with(df, "RULE 061",'CUSTOMER',"WAYNE MUTUAL INSURANCE CO" , "WAYNE MUTUAL INSURANCE CO")

df = apply_rule_starts_with(df, "RULE 062",'CUSTOMER',"ART WATT PAINTING" , "ART WATT PAINTING")
df = apply_rule_starts_with(df, "RULE 063",'CUSTOMER',"BUENA VISTA SECURITY AND PROTE" , "BUENA VISTA SECURITY AND PROTE")
df = apply_rule_starts_with(df, "RULE 064",'CUSTOMER',"GREAT STONE GRANITE" , "GREAT STONE GRANITE")
df = apply_rule_starts_with(df, "RULE 065",'CUSTOMER',"SCHULTHEIS BROS CO" , "SCHULTHEIS BROS CO")
df = apply_rule_starts_with(df, "RULE 066",'CUSTOMER',"PIECE OF GREEN" , "PIECE OF GREEN LANDSCAPING")
df = apply_rule_starts_with(df, "RULE 067",'CUSTOMER',"CLEARPOINT CONSULTING ENGINEER" , "CLEARPOINT CONSULTING ENGINEER")
df = apply_rule_starts_with(df, "RULE 068",'CUSTOMER',"GRADY CRAWFORD CONSTRUCTION CO" , "GRADY CRAWFORD CONSTRUCTION CO")
df = apply_rule_starts_with(df, "RULE 069",'CUSTOMER',"ALLSOUTH APPLIANCE GROUP" , "ALLSOUTH APPLIANCE GROUP")
df = apply_rule_starts_with(df, "RULE 070",'CUSTOMER',"DAC ENTERPRISES" , "DAC ENTERPRISES")
df = apply_rule_starts_with(df, "RULE 071",'CUSTOMER',"HIGHLAND COMMUNITY COLLEGE" , "HIGHLAND COMMUNITY COLLEGE")
df = apply_rule_starts_with(df, "RULE 072",'CUSTOMER',"MATRIX SERVICE COMPANY" , "MATRIX SERVICE COMPANY")
df = apply_rule_starts_with(df, "RULE 073",'CUSTOMER',"TWIN CITY LIMOUSINES" , "TWIN CITY LIMOUSINES")
df = apply_rule_starts_with(df, "RULE 074",'CUSTOMER',"RMI SERVICES CORPORATION" , "RMI SERVICES CORPORATION")
df = apply_rule_starts_with(df, "RULE 075",'CUSTOMER',"GREAT LAKES HOME HEALTH S" , "GREAT LAKES HOME HEALTH S")
df = apply_rule_starts_with(df, "RULE 076",'CUSTOMER',"TOTAL DEPTH" , "TOTAL DEPTH")
df = apply_rule_starts_with(df, "RULE 077",'CUSTOMER',"CAPE BUILDING SYSTEMS" , "CAPE BUILDING SYSTEMS")
df = apply_rule_starts_with(df, "RULE 078",'CUSTOMER',"STATE OF TEXAS" , "STATE OF TEXAS")
df = apply_rule_starts_with(df, "RULE 079",'CUSTOMER',"RICS ELECTRIC" , "RICS ELECTRIC")
df = apply_rule_starts_with(df, "RULE 080",'CUSTOMER',"GILS CARPETS" , "GILS CARPETS")
df = apply_rule_starts_with(df, "RULE 081",'CUSTOMER',"PREMIER THERAPY" , "PREMIER THERAPY")
df = apply_rule_starts_with(df, "RULE 082",'CUSTOMER',"STE GENEVIEVE COUNTY MEMORIAL" , "STE GENEVIEVE COUNTY MEMORIAL")
df = apply_rule_starts_with(df, "RULE 083",'CUSTOMER',"STILSING ELECTRIC INC" , "STILSING ELECTRIC INC")
df = apply_rule_starts_with(df, "RULE 084",'CUSTOMER',"ROBINSON AUTO SALES" , "ROBINSON AUTO SALES")
df = apply_rule_starts_with(df, "RULE 085",'CUSTOMER',"FLORIDA FIRST CALL REMOVAL SER" , "FLORIDA FIRST CALL REMOVAL SER")
df = apply_rule_starts_with(df, "RULE 086",'CUSTOMER',"LANDMARK CONSTRUCTION SOLUTIO" , "LANDMARK CONSTRUCTION SOLUTIO")
df = apply_rule_starts_with(df, "RULE 087",'CUSTOMER',"IMPORT MOTORS OF OLD SAYBROOK" , "IMPORT MOTORS OF OLD SAYBROOK")
df = apply_rule_starts_with(df, "RULE 088",'CUSTOMER',"RALEIGH EAST CONCRETE CONSTRUC" , "RALEIGH EAST CONCRETE CONSTRUC")
df = apply_rule_starts_with(df, "RULE 089",'CUSTOMER',"AMES PLUMBING SERVIC" , "AMES PLUMBING SERVIC")
df = apply_rule_starts_with(df, "RULE 090",'CUSTOMER',"FENIGOR GROUP LLC" , "FENIGOR GROUP LLC")
df = apply_rule_starts_with(df, "RULE 091",'CUSTOMER',"MONTCALM COUNTY ROAD COMMISSIO" , "MONTCALM COUNTY ROAD COMMISSIO")
df = apply_rule_starts_with(df, "RULE 092",'CUSTOMER',"WESTERN INDUSTRIAL CONTRACTORS" , "WESTERN INDUSTRIAL CONTRACTORS")
df = apply_rule_starts_with(df, "RULE 093",'CUSTOMER',"TIRE CORRAL OF AMERICA" , "TIRE CORRAL OF AMERICA")
df = apply_rule_starts_with(df, "RULE 094",'CUSTOMER',"CARVANA" , "CARVANA")

df['EDW_CUSTOMER_NAME'] = df['EDW_CUSTOMER_NAME_ORIGINAL']
df['CUSTOMER_ACCOUNT_NAME'] = df['CUSTOMER_ACCOUNT_NAME_ORIGINAL']

# stupid Tableau can't deal with the addition of a column
# need to figure out how to add this back in
# okay -> turns out this is not down to Tableau, but the stupid way I built the view
del(df['CUST_CALC_RULE'])
del(df['EDW_CUSTOMER_NAME_ORIGINAL'])
del(df['CUSTOMER_ACCOUNT_NAME_ORIGINAL'])

#print(len(df))
#print(len(df.CUSTOMER.unique()))
#df.CUST_CALC_SOURCE.value_counts()

In [10]:
df_j = pd.merge(df, ACCOUNTS_WITH_EBX_PARTY_df, on='CUSTOMER_ACCOUNT_ID', how='left')
df_j.loc[~df_j["PARTY_DEFAULT_NAME"].isnull(),'CUSTOMER'] = df_j.PARTY_DEFAULT_NAME
df_j.loc[~df_j["PARTY_DEFAULT_NAME"].isnull(),'CUST_CALC_SOURCE'] = 'MDM'
df_j.CUST_CALC_SOURCE.value_counts()

del(df_j['PARTY_ID'])
del(df_j['PARTY_DEFAULT_NAME'])

In [13]:
print(len(df_matches_verified))
df_matches_verified.drop_duplicates(subset='CUSTOMER', inplace=True)
print(len(df_matches_verified))

print(len(df_j))
df_j_w_verified = pd.merge(df_j, df_matches_verified, left_on='CUSTOMER', right_on='CUSTOMER', how='left')
print(len(df_j_w_verified))

#df_j_w_verified.loc[~df_j_w_verified["CUSTOMER_CLC"].isnull(),'CUSTOMER'] = df_j_w_verified.CUSTOMER_CLC
#df_j_w_verified.loc[~df_j_w_verified["CUSTOMER_CLC"].isnull(),'CUST_CALC_SOURCE'] = 'CLC'

df_j = df_j_w_verified
#del(df_j['CUSTOMER_CLC'])

472
472
882017
882017


In [14]:
df_j.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,DUNS,IS_BUNDLER,DNB_GLOBAL_ULT_NUMBER,DNB_GLOBAL_ULT_NAME,DNB_DOMESTIC_ULT_NUMBER,DNB_DOMESTIC_ULTIMATE_NAME,DNB_HQ_NUMBER,DNB_HQ_NAME,DNB_DUNS_NUMBER,DNB_BUSINESS_NAME,MDM_WEX_ID,MDM_WEX_NAME,DNB_LEVEL,DNB_CUSTOMER_NAME,EDW_STATE,CUSTOMER,CUST_CALC_SOURCE,MATCH_CUSTOMER_x,DRAW_UP_DATE_x,distance_x,MATCH_CUSTOMER_y,DRAW_UP_DATE_y,distance_y
0,9100008632557,JR AUTO SALES,NaN,47380684,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,47380684,JR AUTO SALES,11951553,JR AUTO SALES,DUNS,JR AUTO SALES,None,JR AUTO SALES,DNB,NaN,NaT,NaN,NaN,NaT,NaN
1,9100008791110,JR AUTO SALES,NaN,47380684,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,47380684,JR AUTO SALES,11951553,JR AUTO SALES,DUNS,JR AUTO SALES,None,JR AUTO SALES,DNB,NaN,NaT,NaN,NaN,NaT,NaN
2,9100007677612,JACK GREENS FLY IN CAMPS LTD,NaN,201449550,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,201449550,GREEN AIRWAYS LTD,<NA>,NaN,DUNS,GREEN AIRWAYS LTD,None,GREEN AIRWAYS LTD,DNB,NaN,NaT,NaN,NaN,NaT,NaN
3,9100009675378,HOLLIS EXTERIORS 2,NaN,67754917,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,67754917,HOLLIS EXTERIORS INC,<NA>,NaN,DUNS,HOLLIS EXTERIORS INC,None,HOLLIS EXTERIORS INC,DNB,NaN,NaT,NaN,NaN,NaT,NaN
4,443008983983,HOLLIS EXTERIORS (2),HOLLIS EXTERIORS (2),67754917,False,<NA>,NaN,<NA>,NaN,<NA>,NaN,67754917,HOLLIS EXTERIORS INC,11843246,HOLLIS EXTERIORS 2,DUNS,HOLLIS EXTERIORS INC,Set,HOLLIS EXTERIORS,EDW,NaN,NaT,NaN,NaN,NaT,NaN


In [15]:
RDW_CONVERSIONS_df.head()

,FLEET_ID,CLASSIC_ACCOUNT_NUMBER,STATUS_INDICATOR,FLEET_NAME,CRDT_LMT_AMT,REP,GROUP_CD,STATUS_DATE,SETUP_DATE,DAYS_SINCE_SETUP,ACCOUNT_AGE
0,9100010226250,0496008303539,Active,Samford University,13000,NaN,NaN,2022-01-25 00:00:00.000,2022-01-25 00:00:00.000,280,10
1,9100010148688,NaN,Terminated,Cayering Lawn Svc.,2400,10914273,NaN,2022-05-20 00:00:00.000,2022-01-04 00:00:00.000,301,10
2,9100007356321,NaN,Terminated,Dale Medical Products Inc (3),2100,NaN,NaN,2022-05-16 00:00:00.000,2019-07-05 00:00:00.000,1215,40
3,9100005259087,0496003709359,Active,Sycamore Volunteer Fire Dept.,1000,NaN,NaN,2022-04-19 00:00:00.000,2018-10-11 00:00:00.000,1482,49
4,6300011064989,NaN,Active,TRANSPORT S. ST-AMOUR,30000,A6F,NaN,2022-05-26 00:00:00.000,2017-03-13 00:00:00.000,2059,68


In [16]:
df_r = RDW_CONVERSIONS_df[['FLEET_ID','CLASSIC_ACCOUNT_NUMBER']].dropna(subset=['CLASSIC_ACCOUNT_NUMBER']).copy()
df_r.FLEET_ID = df_r.FLEET_ID.str.strip()
df_r = df_r[~df_r['FLEET_ID'].str.contains('-',na=False)]
df_r.FLEET_ID = df_r.FLEET_ID.astype('float')
df_r.FLEET_ID = df_r.FLEET_ID.astype('Int64')
print(len(df_r))
print()

df_cust = df_j[['CUSTOMER_ACCOUNT_ID','CUSTOMER']].copy()
df_cust.CUSTOMER_ACCOUNT_ID = df_cust['CUSTOMER_ACCOUNT_ID'].astype('Int64')
print(len(df_cust))
print()

df_rj = pd.merge(df_r, df_cust, left_on='FLEET_ID', right_on='CUSTOMER_ACCOUNT_ID', how='inner')
df_rj = df_rj[pd.to_numeric(df_rj.CLASSIC_ACCOUNT_NUMBER, errors='coerce').notnull()]
df_rj.CLASSIC_ACCOUNT_NUMBER = df_rj.CLASSIC_ACCOUNT_NUMBER.astype(float)
df_rj.CLASSIC_ACCOUNT_NUMBER = df_rj.CLASSIC_ACCOUNT_NUMBER.astype(np.int64)
print(len(df_rj))

df_cust_classic = df_j[['CUSTOMER_ACCOUNT_ID','CUSTOMER']].copy()
df_cust_classic.columns = ['CUSTOMER_ACCOUNT_ID', 'CLASSIC_CUSTOMER']
df_cust_classic.CUSTOMER_ACCOUNT_ID = df_cust['CUSTOMER_ACCOUNT_ID'].astype('Int64')
print(len(df_cust_classic))

df_rj = df_rj[['FLEET_ID','CLASSIC_ACCOUNT_NUMBER','CUSTOMER']]
df_rj = pd.merge(df_rj, df_cust_classic, left_on='CLASSIC_ACCOUNT_NUMBER', right_on='CUSTOMER_ACCOUNT_ID', how='inner')
print(len(df_rj))

df_rdw_conversions = df_rj[df_rj.CUSTOMER!=df_rj.CLASSIC_CUSTOMER]
print(len(df_rdw_conversions))
df_rdw_conversions.head()

df_rdw_conversions = df_rdw_conversions[['CUSTOMER_ACCOUNT_ID','CUSTOMER']]
df_rdw_conversions.columns = ['CUSTOMER_ACCOUNT_ID','CONVERSION_REPLACEMENT_CUSTOMER']
df_rdw_conversions.drop_duplicates(subset=['CUSTOMER_ACCOUNT_ID'], inplace=True)
print(len(df_rdw_conversions))
df_rdw_conversions.head()

733160

882017

310508
882017
50680
29431
29401


,CUSTOMER_ACCOUNT_ID,CONVERSION_REPLACEMENT_CUSTOMER
0,434008301889,CENTURION STONE OF CAROLINAS
1,410004537650,CAR SOLUTION INCORPORATED
2,424007446537,CRESS REFRIGERATION SERVICES INC
3,426007351729,SHARP LAWN AND LANDSCAPING INC
4,444007214588,COLD STORAGE CONSTRUCTION SERVICES INC


In [19]:
print(len(df_j))
df_j_with_rdw_conversions = pd.merge(df_j, df_rdw_conversions, on='CUSTOMER_ACCOUNT_ID',how='left')
print(len(df_j_with_rdw_conversions))

df_j_with_rdw_conversions.loc[~df_j_with_rdw_conversions["CONVERSION_REPLACEMENT_CUSTOMER"].isnull(),'CUSTOMER'] = df_j_with_rdw_conversions.CONVERSION_REPLACEMENT_CUSTOMER
df_j_with_rdw_conversions.loc[~df_j_with_rdw_conversions["CONVERSION_REPLACEMENT_CUSTOMER"].isnull(),'CUST_CALC_SOURCE'] = 'RDW CONVERSIONS'

df_j = df_j_with_rdw_conversions
del(df_j['CONVERSION_REPLACEMENT_CUSTOMER'])

882017
882017


In [20]:
df_j.CUST_CALC_SOURCE.value_counts()

DNB                386350
ACCOUNT            214103
EDW                210400
MDM                 37493
RDW CONVERSIONS     29401
CUSTOM RULE          4270
Name: CUST_CALC_SOURCE, dtype: int64

In [21]:
unique_customer_list = df_j.CUSTOMER.unique()
df_customer_ids = pd.DataFrame(unique_customer_list)
df_customer_ids.columns = ["CUSTOMER"]
df_customer_ids = df_customer_ids.sort_values(['CUSTOMER']).reset_index(drop=True)
df_customer_ids = df_customer_ids.reset_index(drop=False)
df_customer_ids['CUSTOMER_ID'] = df_customer_ids.index + 77000000
del(df_customer_ids['index'])
df_customer_ids.head()

,CUSTOMER,CUSTOMER_ID
0,AIR CONDITION CO,77000000
1,1 AUTO PARTS INC,77000001
2,1109,77000002
3,3D FASTENERS PLUS INC,77000003
4,94775 J AND J SNACK FOODS AA,77000004


In [22]:
print(len(df_j))
df_jj = pd.merge(df_j, df_customer_ids, on='CUSTOMER')
df_jj.dropna(subset=['CUSTOMER'], inplace=True)
print(len(df_jj))
df_jj.head()

882017
882012


,CUSTOMER_ACCOUNT_ID,CUSTOMER_ACCOUNT_NAME,EDW_CUSTOMER_NAME,DUNS,IS_BUNDLER,DNB_GLOBAL_ULT_NUMBER,DNB_GLOBAL_ULT_NAME,DNB_DOMESTIC_ULT_NUMBER,DNB_DOMESTIC_ULTIMATE_NAME,DNB_HQ_NUMBER,DNB_HQ_NAME,DNB_DUNS_NUMBER,DNB_BUSINESS_NAME,MDM_WEX_ID,MDM_WEX_NAME,DNB_LEVEL,DNB_CUSTOMER_NAME,EDW_STATE,CUSTOMER,CUST_CALC_SOURCE,MATCH_CUSTOMER_x,DRAW_UP_DATE_x,distance_x,MATCH_CUSTOMER_y,DRAW_UP_DATE_y,distance_y,CUSTOMER_ID
0,9100008632557,JR AUTO SALES,NaN,47380684,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,47380684,JR AUTO SALES,11951553,JR AUTO SALES,DUNS,JR AUTO SALES,None,JR AUTO SALES,DNB,NaN,NaT,NaN,NaN,NaT,NaN,77325397
1,9100008791110,JR AUTO SALES,NaN,47380684,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,47380684,JR AUTO SALES,11951553,JR AUTO SALES,DUNS,JR AUTO SALES,None,JR AUTO SALES,DNB,NaN,NaT,NaN,NaN,NaT,NaN,77325397
2,9100007677612,JACK GREENS FLY IN CAMPS LTD,NaN,201449550,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,201449550,GREEN AIRWAYS LTD,<NA>,NaN,DUNS,GREEN AIRWAYS LTD,None,GREEN AIRWAYS LTD,DNB,NaN,NaT,NaN,NaN,NaT,NaN,77260186
3,9100009675378,HOLLIS EXTERIORS 2,NaN,67754917,True,<NA>,NaN,<NA>,NaN,<NA>,NaN,67754917,HOLLIS EXTERIORS INC,<NA>,NaN,DUNS,HOLLIS EXTERIORS INC,None,HOLLIS EXTERIORS INC,DNB,NaN,NaT,NaN,NaN,NaT,NaN,77283592
4,443008983983,HOLLIS EXTERIORS (2),HOLLIS EXTERIORS (2),67754917,False,<NA>,NaN,<NA>,NaN,<NA>,NaN,67754917,HOLLIS EXTERIORS INC,11843246,HOLLIS EXTERIORS 2,DUNS,HOLLIS EXTERIORS INC,Set,HOLLIS EXTERIORS INC,RDW CONVERSIONS,NaN,NaT,NaN,NaN,NaT,NaN,77283592


In [23]:
df_by_account = df_jj[['CUSTOMER_ACCOUNT_ID','CUSTOMER_ID', 'CUSTOMER']]
df_by_account.head()

,CUSTOMER_ACCOUNT_ID,CUSTOMER_ID,CUSTOMER
0,9100008632557,77325397,JR AUTO SALES
1,9100008791110,77325397,JR AUTO SALES
2,9100007677612,77260186,GREEN AIRWAYS LTD
3,9100009675378,77283592,HOLLIS EXTERIORS INC
4,443008983983,77283592,HOLLIS EXTERIORS INC


In [24]:
len(df_by_account.CUSTOMER_ID.unique())

653746

In [25]:
df_jj.CUST_CALC_SOURCE.value_counts()

DNB                386350
ACCOUNT            214098
EDW                210400
MDM                 37493
RDW CONVERSIONS     29401
CUSTOM RULE          4270
Name: CUST_CALC_SOURCE, dtype: int64

In [26]:
print(len(df_jj))
print(len(ACCOUNT_NEW_SALES_df))
ACCOUNT_NEW_SALES_df['HAS_SALES_FLAG'] = True
df_j_with_sales = pd.merge(df_jj, ACCOUNT_NEW_SALES_df, on='CUSTOMER_ACCOUNT_ID', how='left')
print(len(df_j_with_sales))

882012
153734
882012


In [27]:
df_j_with_sales.HAS_SALES_FLAG.value_counts(dropna=False)

NaN     728280
True    153732
Name: HAS_SALES_FLAG, dtype: int64

In [28]:
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df = df_j_with_sales

# Write recipe outputs
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS = dataiku.Dataset("ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS")
ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS.write_with_schema(ACCOUNTS_WITH_CUSTOMER_FROM_EDW_AND_DUNS_df)

BY_ACCOUNT = dataiku.Dataset("BY_ACCOUNT")
BY_ACCOUNT.write_with_schema(df_by_account)

/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:437: ResourceWarning: unclosed <socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 48728), raddr=('10.92.106.166', 10001)>
  return _handle_void_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)
/opt/dataiku-dss-10.0.5/python/dataiku/core/intercom.py:413: ResourceWarning: unclosed <socket.socket fd=59, family=AddressFamily.AF_INET, type=SocketKind.SOCK_STREAM, proto=6, laddr=('10.92.106.166', 48730), raddr=('10.92.106.166', 10001)>
  return _handle_json_resp(backend_api_post_call(path, data, **kwargs), err_msg = err_msg)


KeyboardInterrupt: 